In [2]:
!pip install vsts

In [3]:
from vsts.vss_connection import VssConnection
from msrest.authentication import BasicAuthentication

token='
team_instance='https://mseng.visualstudio.com'
credentials = BasicAuthentication('', token)
connection = VssConnection(base_url=team_instance, creds=credentials)
git_client = connection.get_client('vsts.git.v4_1.git_client_base.GitClientBase')

In [7]:
%%time
from vsts.git.v4_1.models.git_pull_request_search_criteria import GitPullRequestSearchCriteria

'''
    Get the pull request data from the master branch of vsoonline repo
    we are interested in the title / description / list of filenames / workitemid. 
'''

class PullRequest:         
    def __init__(self, title, description, pull_request_id, wits=[], files = []):
        self.title = title
        self.description = description
        self.id = pull_request_id
        self.files = files
        self.wits = wits
        
# make sure this something like 100*k + 1
start_id = 1
# make sure this is always a multiple of 100
end_id = 15
# default batch size
batch_size = 1000
#project id : b924d696-3eae-4116-8443-9a18392d8544
#git repo id : fb240610-b309-4925-8502-65ff76312c40
#getting all the pull requests....
total_count = 0
search_param = GitPullRequestSearchCriteria(status='all')
project_id='b924d696-3eae-4116-8443-9a18392d8544'
repo_id='fb240610-b309-4925-8502-65ff76312c40'
all_prs = []
while (start_id <= end_id) :
    pull_requests =  git_client.get_pull_requests(repo_id,search_param,max_comment_length=0, skip=start_id,top=start_id+batch_size)
    for pr in pull_requests:
        #get list of files. 
        files = git_client.get_attachments(repo_id,pr.pull_request_id)
        #get linked workitems.
        workitems = git_client.get_pull_request_work_item_refs(repo_id,pr.pull_request_id)
        all_prs.append(PullRequest(pr.title, pr.description, pr.pull_request_id,workitems, files))
    start_id += batch_size


0
Wall time: 3min 28s
